In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('inputs/train.csv', index_col='id')
data['date'] = pd.to_datetime(data['date'])

stores = pd.read_csv('inputs/stores.csv')
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [2]:
data_stores = data.merge(stores, left_on='store_nbr', right_on='store_nbr')

In [3]:
transact = pd.read_csv('inputs/transactions.csv')
transact['date'] = pd.to_datetime(transact['date'])
data_stores_trans  = data_stores.merge(transact, on=['store_nbr', 'date'])

In [4]:
data_stores_trans.head()

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions
0,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,2111
1,2013-01-02,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,2111
2,2013-01-02,1,BEAUTY,2.0,0,Quito,Pichincha,D,13,2111
3,2013-01-02,1,BEVERAGES,1091.0,0,Quito,Pichincha,D,13,2111
4,2013-01-02,1,BOOKS,0.0,0,Quito,Pichincha,D,13,2111


In [5]:
df = data_stores_trans.copy()

In [6]:
# df['sales'].replace(0, np.nan, inplace=True)

In [7]:
# df.groupby(['store_nbr']).median().sort_values('transactions')

THE CODE BELOW INTEGRATES THE HW4 STUFF AND ADDS THE WORK FROM ABOVE...

In [8]:
# path = 'inputs/'
# # read oil price
# data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
#
# ########################################################################################################################
# # TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# # Hint: check the documentation of .rolling() method of pandas.DataFrame
# ########################################################################################################################
# data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean()
#
# # Create continguous moving average of oil prices
# calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))
#
# ########################################################################################################################
# # TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# # as calendar.
# # TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# # Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
# ########################################################################################################################
# calendar = calendar.join(data_oil)
#
# data_oil['ma_oil'].fillna(method='ffill', inplace=True)
# calendar['ma_oil'].fillna(method='ffill', inplace=True)

In [9]:
data = pd.read_csv('inputs/train.csv', index_col='id')
data['date'] = pd.to_datetime(data['date'])
data['nat_holiday'] = 0
data['reg_holiday'] = 0
data['loc_holiday'] = 0

stores = pd.read_csv('inputs/stores.csv', index_col='store_nbr')

holidays = pd.read_csv('inputs/holidays_events.csv')
holidays['date'] = pd.to_datetime(holidays['date'])

holidays = holidays.reset_index()
stores = stores.reset_index()

for index, row in holidays.iterrows():
    store_nbr_list = []
    if row['locale'] == 'Local':
        for sindex, srow in stores.iterrows():
            if srow['city'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'loc_holiday'] = 1
    if row['locale'] == 'Regional':
        for sindex, srow in stores.iterrows():
            if srow['state'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'reg_holiday'] = 1
    if row['locale'] == 'National':
        data.loc[data['date'] == row['date'], 'nat_holiday'] = 1

In [10]:
path = 'inputs/'
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True)

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean()

# Create continguous moving average of oil prices
calendar = pd.DataFrame()
calendar['date'] = pd.date_range('2013-01-01', '2017-08-31')
# calendar.head()
########################################################################################################################
# TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# as calendar.
# TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
########################################################################################################################
calendar = calendar.merge(data_oil, on='date')
calendar.date = calendar.date.dt.to_period('D')

# calendar.head()

data_oil['ma_oil'].fillna(method='ffill', inplace=True)
calendar['ma_oil'].fillna(method='ffill', inplace=True)

In [11]:
calendar['wd'] = [i.dayofweek < 5 for i in calendar['date']]

In [12]:
stores_new = pd.read_csv(path + 'stores.csv',
                         usecols=['store_nbr', 'city', 'state', 'type', 'cluster'],
                         dtype={'store_nbr': 'category', 'city': 'category', 'state': 'category', 'type': 'category', 'cluster': 'category'})

In [13]:
transactions = pd.read_csv(path + 'transactions.csv',
                          usecols=['store_nbr', 'date', 'transactions'],
                          dtype={'store_nbr': 'category', 'transactions': 'float32'},
                          parse_dates=['date'], infer_datetime_format=True)
transactions.date = transactions.date.dt.to_period('D')

In [14]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
# df_train = temp.merge(transactions, on=['store_nbr','date'])


In [15]:
temp = df_train.merge(transactions, on =['store_nbr', 'date'])

In [16]:
temp2 = temp.merge(stores_new, on='store_nbr')

In [17]:
df_train = temp2.copy()
# df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

In [18]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
# Adding in additional features


df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [19]:
store1 = df_train.loc[df_train['store_nbr'] == '1']
# df_train.head()
# df_train.groupby('store_nbr')
# ddf = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

In [20]:
testy = df_train.merge(calendar, on='date')
testy.head()

,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster,dcoilwtico,ma_oil,wd
0,2013-01-01,25,AUTOMOTIVE,0.0,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True
1,2013-01-01,25,BABY CARE,0.0,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True
2,2013-01-01,25,BEAUTY,2.0,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True
3,2013-01-01,25,BEVERAGES,810.0,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True
4,2013-01-01,25,BOOKS,0.0,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True


In [21]:
# data = pd.read_csv('inputs/train.csv', index_col='id')
# data['date'] = pd.to_datetime(data['date'])
data = testy.copy()
data['nat_holiday'] = 0
data['reg_holiday'] = 0
data['loc_holiday'] = 0


# stores = pd.read_csv('inputs/stores.csv', index_col='store_nbr')
stores = stores_new.copy()
# holidays = pd.read_csv('inputs/holidays_events.csv')
# holidays['date'] = pd.to_datetime(holidays['date'])
holidays = pd.read_csv(path + 'holidays_events.csv',
                       usecols=['date', 'type', 'locale', 'locale_name', 'transferred'],
                       dtype={'type': 'category', 'locale': 'category', 'locale_name': 'category', 'transferred': 'bool'},
                       parse_dates=['date'], infer_datetime_format=True
                       )

holidays.date = holidays.date.dt.to_period('D')
holidays = holidays.reset_index()
stores = stores.reset_index()

for index, row in holidays.iterrows():
    store_nbr_list = []
    if row['locale'] == 'Local':
        for sindex, srow in stores.iterrows():
            if srow['city'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'loc_holiday'] = 1
    if row['locale'] == 'Regional':
        for sindex, srow in stores.iterrows():
            if srow['state'] == row['locale_name']:
                data.loc[(data['date'] == row['date']) & (data['store_nbr'] == srow['store_nbr']), 'reg_holiday'] = 1
    if row['locale'] == 'National':
        data.loc[data['date'] == row['date'], 'nat_holiday'] = 1

In [22]:
data

,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster,dcoilwtico,ma_oil,wd,nat_holiday,reg_holiday,loc_holiday
0,2013-01-01,25,AUTOMOTIVE,0.000000,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
1,2013-01-01,25,BABY CARE,0.000000,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
2,2013-01-01,25,BEAUTY,2.000000,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
3,2013-01-01,25,BEVERAGES,810.000000,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
4,2013-01-01,25,BOOKS,0.000000,0.0,770.0,Salinas,Santa Elena,D,1,NaN,NaN,True,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1968478,2016-10-26,22,POULTRY,206.367004,0.0,633.0,Puyo,Pastaza,C,7,48.75,50.17,True,0,0,0
1968479,2016-10-26,22,PREPARED FOODS,37.000000,10.0,633.0,Puyo,Pastaza,C,7,48.75,50.17,True,0,0,0
1968480,2016-10-26,22,PRODUCE,277.283997,4.0,633.0,Puyo,Pastaza,C,7,48.75,50.17,True,0,0,0
1968481,2016-10-26,22,SCHOOL AND OFFICE SUPPLIES,0.000000,0.0,633.0,Puyo,Pastaza,C,7,48.75,50.17,True,0,0,0
